In [8]:
from selenium import webdriver 
import pandas as pd 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
import time
import re
from bs4 import BeautifulSoup

In [9]:
yt_link = VIDEO_LINK = input()
# yt_link = VIDEO_LINK = 'https://www.youtube.com/watch?v=x_VrgWTKkiM'

In [10]:
def comment_scraper(yt_link):
    
    driver = webdriver.Chrome()

    driver.get(yt_link)
    
    SCROLL_PAUSE_TIME = 5

    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    counter = 0
    
    while True:
        # Scroll down to bottom
        driver.execute_script(f"window.scrollTo({last_height},document.documentElement.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        
        driver.execute_script(f"scrollBy({last_height},-1000);")
        
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        
        if new_height == last_height:
            print("break")
            break
        last_height = new_height

    
    comment_div = driver.find_element_by_xpath('//*[@id="contents"]')

    buttons = driver.find_elements_by_xpath('//*[@id="more-replies"]/a')

    comments = comment_div.find_elements_by_xpath('//*[@id="content-text"]')
    
    replies_elems =driver.find_elements_by_xpath('//*[@id="replies"]')
    
    all_replies = [elem.text for elem in replies_elems]

    name_elems = driver.find_elements_by_xpath('//*[@id="author-text"]')

    likes = comment_div.find_elements_by_xpath('//*[@id="vote-count-middle"]')
    
    comments_scraped_wo_reps = {'comment_' + str(idx): (name.text, comment.text, likes.text)\
                                for idx, (name, comment, likes) in enumerate(zip(name_elems, comments, likes), 1)}
    
    
    for button in buttons:
        time.sleep(2)
        webdriver.ActionChains(driver).move_to_element(button).click(button).perform()

    replies = driver.find_elements_by_xpath('//*[@id="loaded-replies"]/ytd-comment-renderer')
    replies_scrapped = {f'reply_{idx}': reply.text for idx, reply in enumerate(replies, 1) if reply.text != ''}
    comments = comment_div.find_elements_by_xpath('//*[@id="content-text"]')
    comments_txt = [comment.text for comment in comments]
    name_elems = driver.find_elements_by_xpath('//*[@id="author-text"]')
    name_txt = [name.text for name in name_elems]
    likes = driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')
    likes_txt = [like.text for like in likes]
    
    
    
    driver.close()
    
    comments_scraped_with_reps = {'comment_' + str(idx): (name, comment, likes)\
                        for idx, (name, comment, likes) in enumerate(zip(name_txt, comments_txt, likes_txt), 1)}

    return comments_scraped_with_reps, comments_scraped_wo_reps,all_replies

In [11]:
def video_info_scraper(yt_link):
    
    driver = webdriver.Chrome()

    driver.get(yt_link)
    
    time.sleep(2)

    driver.execute_script("window.scrollTo(0,document.documentElement.scrollHeight);")

    time.sleep(2)

    driver.execute_script("scrollBy(0,-1000);")

    time.sleep(5)
    
    views = driver.find_element_by_xpath('//yt-view-count-renderer').text.split(' ')[0]
    
    title = driver.find_element_by_css_selector('.title').text

    num_comments = float(driver.find_element_by_xpath('//ytd-comments-header-renderer').text.split(' ')[0])

    num_likes_dislikes = driver.find_elements_by_xpath('//ytd-toggle-button-renderer')[0:2]

    num_likes = num_likes_dislikes[0].text

    num_dislikes = num_likes_dislikes[1].text
    
    scraping_incomplete = True
    
    driver.close()
        
    while scraping_incomplete:
        
        comments_scraped_with_reps, comments_scraped_wo_reps,all_replies = comment_scraper(yt_link)
        
        if float(len(comments_scraped_with_reps)) == num_comments:
            
            scraping_incomplete = False
            
    return {'video_title': title, 'num_views': views, 'num_comments': num_comments,
            'num_likes': num_likes, 'num_dislikes': num_dislikes,
            'comments_scraped_wo_reps': comments_scraped_wo_reps,
            'comments_scraped_with_reps': comments_scraped_with_reps,
           'all_replies':all_replies}

In [12]:
def convert_values_to_list(comm):
    for k,v in sorted(comm.items()):
            comm[k]=list(comm[k])
    return comm

In [13]:
def number_of_replies(all_replies,comments_scraped_wo_reps):
    import re
    for count in range(0,len(all_replies)):
        if all_replies[count] is '':
            all_replies[count]=0 
            
    for i in range(0,len(all_replies)):
        num_replies=re.findall(r'\d+',str(all_replies[i]))
        if len(num_replies) is 0:
            num_replies.append(1)
        x=list(comments_scraped_with_reps)[i]
        comments_scraped_wo_reps[x].append(num_replies[0])
    
    return comments_scraped_wo_reps

In [14]:
def mapping(cwor,cmr):
    for k,v in (cwor.items()):
        reply=v[3]
        name=v[0]

        for key,value in (cmr.items()):
            if str(value[0]) == str(name):
                next_key=key
                key_index=list(cmr).index(next_key)
            
                break
        
        reply_com=[]       
        for i in range(1,int(reply)+1):
            next_ind=list(cmr)[key_index+i]
            reply_com.append(cmr[next_ind])
        
        v.append(reply_com)
        
    return cwor

In [15]:
vedio_info = video_info_scraper(VIDEO_LINK)
comments_scraped_with_reps, comments_scraped_wo_reps,all_replies=comment_scraper(VIDEO_LINK)

break
break


In [16]:
comments_scraped_with_reps=convert_values_to_list(comments_scraped_with_reps)
comments_scraped_wo_reps=convert_values_to_list(comments_scraped_wo_reps)

In [17]:
comments_scraped_wo_reps=number_of_replies(all_replies,comments_scraped_wo_reps)

In [18]:
comment_with_replies=mapping(comments_scraped_wo_reps,comments_scraped_with_reps)

In [19]:
comment_with_replies

{'comment_1': ['Victor zetterström',
  'This series is really amazing! Love it!',
  '',
  1,
  [['Laurence Moroney', 'Awesome, thanks!', '1']]],
 'comment_2': ['antonio rizzo',
  'Bravo!!! So far the best explanation and support material on NN and DL',
  '',
  '0',
  []],
 'comment_3': ['Animesh Wakade',
  "Seymore I'm loving it.. FEED me the next ep!!",
  '',
  1,
  [['Laurence Moroney',
    "I'm a mean green mother from Western Europe and I'm fightin' mad! :)",
    '']]],
 'comment_4': ['Patrick Jähne',
  "I love this series of explaination! It's easy to follow along. I can't wait for the next episode!",
  '',
  1,
  [['Laurence Moroney', 'Next one is last one :)', '']]],
 'comment_5': ['Jousef Murad',
  'Laurence is the best one in explaining these things! Laurence "TheFeynman" Moroney :D',
  '2',
  1,
  [['Laurence Moroney', 'Haha! The Feynman! :)', '1']]],
 'comment_6': ['Frederik Steinmetz',
  "I do have a follow-up question and I have spent days to figure it out. I used your exa

In [20]:
len(comments_scraped_with_reps)

104